In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
# import osmnx as ox
import numpy as np


# import sys
# sys.path.append('/Users/danielsuarez/Documents/Academic/Spring2025/SeniorDesign/CMOR492-DWS/')


from network_construction.network import source_treatment, get_Utown



In [2]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G, 40)  # <-- Specify # starting points for treatment node algorithm

In [3]:
### MODEL PARAMETERS

# TODO: What if we just make pipe size continuous/linear

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


# D = [0.25, 0.5, 1]  # Pipe diameters
# CP = {0.25: 151, 0.5: 220, 1:450}  # Cost per unit of pipe


SR = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = 0.05
    SR[node] = 0.05

total_flow = sum(SR.values())

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 1000
    CAP[node] = 1000

Vmin = 0.6
Vmax = 3

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 1000000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width


In [10]:
m = gp.Model()

### DECISION VARIABLES

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name=[f'x: {i} --> {j}' for i, j in Path.keys()])  # Path ij used
y = m.addVars(treatment_nodes, vtype=GRB.BINARY, name=[f'y_{j}' for j in treatment_nodes])  # treatment at node j
z = m.addVars(G.edges, vtype=GRB.BINARY)  # edge e used

# Pipe Size Selection
d_es_names = []
for e in G.edges:
    for s in D:
        d_es_names.append(f"Edge: {e[0]}---{e[1]} <- Size: {s}")

d = m.addVars(G.edges, D, vtype=GRB.BINARY, name=d_es_names)  # Pipe size s at edge e


# Recourse Amount
r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'r_{i}' for i in source_nodes])  # flow handled at trucking at edge e

# Flow in Edge e
Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'q_{e}' for e in G.edges])  # Flow in Edge e

# Node Elevation
el = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, name=[f'el_{v}' for v in G.nodes])  # Elevation at node el_v

# Path Flow
p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name=[f"p: {i} --> {j}" for i, j in Path.keys()])

m.update()

In [11]:
### CONSTRAINTS

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs((p[i, j] >= (SR[i] * x[i, j]) - r[i] for i, j in Path.keys()), name='node_prod_rec')

# TREATMENT CAPACITY
treat_cap = m.addConstrs((gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j] for j in treatment_nodes), name='treat_cap')

#  NODE ASSIGNMENT
node_assign = m.addConstrs((gp.quicksum(x[i, j] for j in treatment_nodes) == 1 for i in source_nodes), name='node_assign')

# PIPE SIZING
pipe_sizing = m.addConstrs((gp.quicksum(d[*e, s] for s in D) == z[e] for e in G.edges), name='pipe_sizing')  # ALWAYS BE SURE TO EXPAND e

# TODO: Go through this with John
# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs((Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges), name='flow_def')


# MIN/MAX SLOPE
min_slope = m.addConstrs((el[e[0]] - el[e[1]] >= (LE[e] * Smin) - (M * (1 - z[e])) for e in G.edges), name='min_slope')
max_slope = m.addConstrs((el[e[0]] - el[e[1]] <= (LE[e] * Smax) + (M * (1 - z[e])) for e in G.edges), name='max_slope')

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs((Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (d[*e, s]) for s in D) for e in G.edges), name='flow_vel')

# PIPES UNDERGROUND
underground = m.addConstrs((el[i] <= EL[i] for i in source_nodes), name='underground')
m.update()

In [12]:
# EDGE ACTIVATION

# TODO: Go through this with John 2
# EDGE ACTIVATION
ePath = {}  # Use this for Edge Activiation Constraint
for e, p in Path.items():
    ePath[e] = [(p[l - 1], p[l]) for l in range(1, len(p))]

edge_activate = m.addConstrs((gp.quicksum(z[e] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j] for i, j in Path), name='edge_activate')

In [13]:
# ENVELOPES FOR MANNING

T = 11.9879
P = lambda LE, s: LE / (T * (s**(16/3)))
Qmax = lambda s: Vmax * ((np.pi / 8) * (s**2))


alpha = m.addVars(G.edges, D, lb=0, name='alpha')
beta = m.addVars(G.edges, D, lb=0, name='beta')


alpha_2 = m.addConstrs((alpha[*e, s] >= Q[e] + d[*e, s] * Qmax(s) - ( Qmax(s)) for e in G.edges for s in D), name='alpha_2')
alpha_3 = m.addConstrs((alpha[*e, s] <= Qmax(s) * d[*e, s] for e in G.edges for s in D), name='alpha_3')
alpha_4 = m.addConstrs((alpha[*e, s] <= Q[e] for e in G.edges for s in D), name='alpha_4')
alpha_5 = m.addConstrs((alpha[*e, s] <= Qmax(s) for e in G.edges for s in D), name='alpha_5')

beta_2 = m.addConstrs((beta[*e, s] >= (Qmax(s) * Q[e]) + (Qmax(s) * alpha[*e, s]) - (Qmax(s)**2) for e in G.edges for s in D), name='beta_2')
beta_3 = m.addConstrs((beta[*e, s] <= Qmax(s) * alpha[*e, s] for e in G.edges for s in D), name='beta_3')
beta_4 = m.addConstrs((beta[*e, s] <= Qmax(s) * Q[e] for e in G.edges for s in D), name='beta_4')

# manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= 0 for e in G.edges), name='manning_2')

# ADDED THE BIG M THING HERE BUT IDK IF IT COULD BE IMPROVED
manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= (1-z[e])*M for e in G.edges), name='manning_2')
m.update()

In [ ]:
# # MANNING'S EQUATION
#
# # manning = m.addConstrs(
# #     (LE[u, v, 0] * gp.quicksum(d_es[u, v, 0, s] * ((Q[u, v, 0]**2) / ((11.9879 * (s**(8/3)))**2)) for s in D)
# #      <= el[u] - el[v] for u, v, _ in G.edges),
# #     name="manning")
#

# JOHN REFORMULATION
# manning = m.addConstrs((LE[u, v, 0] * ((Q[u, v, 0] / (11.9879 * (s**2.6666667)))**2) <=
#                         el[u] - el[v] + (M * (1 - d_es[u, v, 0, s])) for u, v, _ in G.edges for s in D),
#                        name="manning")

# m.update()

In [14]:
## OBJECTIVE EPXR 1: TREATMENT COSTS

treat_cost = gp.LinExpr()
for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j])

# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u]) + (EL[v] - el[v])) / 2) * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))

# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost_f = lambda u, v: gp.LinExpr(CB * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))
# OBJECTIVE EXPR 4: PIPE COSTS
pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v] * gp.quicksum(CP[s] * d[u, v, s] for s in D))

excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v in G.edges)

# OBJECTIVE EXPR 5: RECOURSE TRUCKING

rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])

m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

# m.setObjective(0, GRB.MINIMIZE)

m.update()
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")

Model has 20638 variables and 34748 constraints.


In [15]:
m.write("singleperiod_nocontext.lp")

In [9]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 34748 rows, 20638 columns and 223307 nonzeros
Model fingerprint: 0x7b89d9bb
Variable types: 12003 continuous, 8635 integer (8635 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+06]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 1e+06]
Presolve removed 23543 rows and 8636 columns
Presolve time: 1.36s
Presolved: 11205 rows, 12002 columns, 109465 nonzeros
Variable types: 7280 continuous, 4722 integer (4722 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 1.58 seconds (0.74 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.

In [ ]:
for v in r:
    if r[v].X > 0:
        print(r[v].VarName, r[v].X)

In [ ]:
m.computeIIS()

for c in m.getConstrs():
    if c.IISConstr:
        print(f"Constraint {c.ConstrName} is in the IIS")

for v in m.getVars():
    if v.IISLB > 0:
        print(f"Lower bound of {v.VarName} is in the IIS")
    elif v.IISUB > 0:
        print(f"Upper bound of {v.VarName} is in the IIS")
